In [ ]:
!wget https://technionmail-my.sharepoint.com/:u:/g/personal/ploznik_campus_technion_ac_il/EQc79uRBeO1FqtH6ILFDx78BuuWui3DuRaBtnzTB6Aqxqg?download=1 -O data.tar
!tar -xvf data.tar

In [2]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [13]:
df_test_check = pd.read_csv('/content/len_of_test.csv')

In [24]:
df_test_check['file_len'].sum()

385849

In [16]:
df_test_check[df_test_check['diff_of_sepsis']<1].shape


(105, 3)

In [4]:
import plotly.graph_objs as go
import plotly.offline as pyo
from scipy.stats import ttest_ind


In [ ]:
df_train = pd.DataFrame()
df_test = pd.DataFrame()
for directory in ["/content/data/train", "/content/data/test"]:
  for filename in os.listdir(directory):
    if filename.endswith('.psv'): 
      # Load the data from the PSV file into a dataframe
      file_path = os.path.join(directory, filename)
      file_data = pd.read_csv(file_path, sep='|')
      file_data['patient'] = str(filename).split('.')[0].split('_')[1]
      if "train" in directory:
      # Append the data to the main dataframe
        index_up_to = len(file_data) - len(file_data[file_data['SepsisLabel'] == 1])
        file_data = file_data[file_data.index<=index_up_to]
        df_train = pd.concat([df_train, file_data])
      else: 
        df_test = pd.concat([df_test, file_data])

In [3]:
df_train = pd.read_csv("/content/trainset.csv")


In [6]:
count_by_patient = df_train.groupby('patient').count()

In [13]:
((count_by_patient > 0).sum() / 20000)

HR                  0.98980
O2Sat               0.98920
Temp                0.98025
SBP                 0.98245
MAP                 0.98725
DBP                 0.81110
Resp                0.98795
EtCO2               0.07450
BaseExcess          0.31915
HCO3                0.48635
FiO2                0.42655
pH                  0.45505
PaCO2               0.44000
SaO2                0.31220
AST                 0.34040
BUN                 0.93410
Alkalinephos        0.33535
Calcium             0.85025
Chloride            0.51560
Creatinine          0.93300
Bilirubin_direct    0.04760
Glucose             0.94715
Lactate             0.29065
Magnesium           0.86160
Phosphate           0.68505
Potassium           0.93850
Bilirubin_total     0.33870
TroponinI           0.17260
Hct                 0.92810
Hgb                 0.92385
PTT                 0.46435
WBC                 0.91965
Fibrinogen          0.09885
Platelets           0.92015
Age                 0.99010
Gender              

In [8]:
count_by_patient.mean()

HR                  34.105999
O2Sat               32.904808
Temp                12.745026
SBP                 32.309615
MAP                 33.122513
DBP                 26.064286
Resp                32.014544
EtCO2                1.312898
BaseExcess           1.974447
HCO3                 1.489850
FiO2                 2.995758
pH                   2.536713
PaCO2                2.020351
SaO2                 1.285830
AST                  0.588829
BUN                  2.512877
Alkalinephos         0.582567
Calcium              2.186799
Chloride             1.623220
Creatinine           2.242955
Bilirubin_direct     0.069084
Glucose              6.446268
Lactate              0.961873
Magnesium            2.349207
Phosphate            1.482073
Potassium            3.450965
Bilirubin_total      0.544137
TroponinI            0.365014
Hct                  3.267397
Hgb                  2.708969
PTT                  1.039188
WBC                  2.347440
Fibrinogen           0.227098
Platelets 

In [1]:
df_train = pd.read_csv("/content/trainset.csv")
df_test = pd.read_csv("/content/testset.csv")

NameError: ignored

In [17]:
df_test = pd.read_csv("/content/testset.csv")

In [22]:
df_test.groupby('patient').mean()[df_test.groupby('patient').mean()['SepsisLabel']>0].shape

(741, 41)

In [11]:
df_test[df_test['patient']==10]['HR'].iloc[:5].mean()

85.6

In [9]:
mean_by_patient = df_train.groupby('patient').mean()

In [14]:
count_by_patient = df_train.groupby('patient').count()

In [28]:
mean_by_patient_sick = mean_by_patient[mean_by_patient['SepsisLabel'] > 0]
mean_by_patient_healthy = mean_by_patient[mean_by_patient['SepsisLabel'] == 0]

T test to determine if there is a difference in the mean age between the sick and healthy patients

In [79]:
t, p = ttest_ind(mean_by_patient_sick['Age'], mean_by_patient_healthy['Age'], equal_var=False)

# Print the results
print(f"t-value: {t:.3f}")
print(f"p-value: {p:.3f}")

if p < 0.05:
    print("The mean age of the sick patients is significantly different from the mean age of the healthy patients.")
else:
    print("There is no significant difference in the mean age between the sick and healthy patients.")

t-value: 1.322
p-value: 0.186
There is no significant difference in the mean age between the sick and healthy patients.


In [84]:
X_sick = mean_by_patient_sick['Temp'].copy()
X_sick.fillna(X_sick.mean(), inplace=True)

X_healthy = mean_by_patient_healthy['Temp'].copy()
X_healthy.fillna(X_healthy.mean(), inplace=True)

In [87]:
np.mean(X_sick)

37.02335969156118

In [88]:
np.mean(X_healthy)

36.835154906995555

In [85]:
t, p = ttest_ind(X_sick, X_healthy, equal_var=False)

# Print the results
print(f"t-value: {t:.3f}")
print(f"p-value: {p:.3f}")

if p < 0.05:
    print("The mean Temp of the sick patients is significantly different from the mean Temp of the healthy patients.")
else:
    print("There is no significant difference in the mean Temp between the sick and healthy patients.")

t-value: 9.228
p-value: 0.000
The mean Temp of the sick patients is significantly different from the mean Temp of the healthy patients.


In [108]:
def histogram_of_mean_values(colname, vert_line=-1):
  trace1 = go.Histogram(x=mean_by_patient_sick[colname], name='Sick', histnorm='percent')
  trace2 = go.Histogram(x=mean_by_patient_healthy[colname], name='Healthy', histnorm='percent')
  layout = go.Layout(title=f'Histograms of {colname}',
                    xaxis=dict(title=f'{colname}'),
                    yaxis=dict(title='Percentage of patients'))
  data = [trace1, trace2]
  fig = go.Figure(data=data, layout=layout)
  if vert_line > -1:
    fig.add_shape(type="line", x0=vert_line, y0=0, x1=vert_line, y1=4, line=dict(color="black", width=3))
    fig.add_annotation(x=vert_line+6, y=3.25, text="22 Breaths per minute", showarrow=False, font=dict(size=16))

  pyo.iplot(fig, filename='histograms.html')

In [48]:
histogram_of_mean_values('Age')

In [49]:
histogram_of_mean_values('Temp')

In [109]:
histogram_of_mean_values('Resp', 22)

In [67]:
histogram_of_mean_values('Glucose')

In [50]:
mean_by_patient_sick

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,FiO2,pH,PaCO2,SaO2,AST,BUN,Alkalinephos,Calcium,Chloride,Creatinine,Bilirubin_direct,Glucose,Lactate,Magnesium,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,SepsisLabel
patient,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,88.173913,100.000000,37.754286,136.500000,87.260870,63.260870,16.052632,NaN,0.500000,23.5,0.400000,7.425000,36.500000,NaN,NaN,16.000000,NaN,8.300000,109.000000,1.000000,NaN,138.750000,1.100000,2.300000,4.050000,4.100000,NaN,NaN,33.400000,12.250000,25.600,11.550000,NaN,205.000000,65.79,1.0,0.041667
17,108.776119,96.181818,36.811111,108.689394,77.416667,59.931818,20.277778,29.581395,NaN,NaN,0.458571,7.430000,31.000000,97.960,368.0,65.714286,80.333333,6.366667,NaN,2.465714,NaN,134.750000,2.610000,2.300000,4.300000,4.165000,1.033333,NaN,30.550000,9.975000,NaN,10.050000,NaN,103.500000,89.00,1.0,0.012500
20,100.018367,98.451327,36.709375,120.692623,81.316872,67.448770,16.868644,NaN,NaN,NaN,0.633333,7.385000,39.650000,95.975,21.0,25.750000,72.666667,7.755625,NaN,8.030833,NaN,125.916667,3.052500,2.325000,3.880000,5.023125,0.766667,0.750909,31.916667,10.950000,113.075,8.866667,201.000000,156.333333,56.00,0.0,0.003774
54,79.479167,97.500000,38.192500,114.645833,80.041667,68.604167,22.000000,18.000000,NaN,NaN,0.475000,7.427500,37.050000,97.650,NaN,13.000000,NaN,3.058000,NaN,1.100000,NaN,138.000000,5.237500,1.900000,NaN,4.420000,NaN,NaN,26.300000,8.900000,32.400,11.300000,NaN,113.000000,45.00,1.0,0.038462
56,71.297101,95.384058,37.138889,133.152174,85.927536,59.253623,18.704545,NaN,NaN,NaN,0.500000,NaN,NaN,NaN,13.0,5.000000,71.000000,7.333333,NaN,0.513333,0.1,119.285714,1.470000,2.366667,2.333333,3.966667,0.300000,0.010000,32.650000,10.275000,32.600,13.725000,NaN,333.500000,60.00,0.0,0.012821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19949,50.687500,95.312500,35.805000,99.375000,64.020000,NaN,13.750000,NaN,NaN,21.0,NaN,NaN,NaN,NaN,20.0,37.000000,272.000000,7.100000,96.000000,1.800000,NaN,173.000000,NaN,3.300000,6.200000,4.500000,0.100000,NaN,38.500000,12.200000,35.000,43.600000,NaN,225.000000,68.67,0.0,0.125000
19957,59.767857,99.839286,37.134286,128.732143,89.785714,70.589286,14.107143,NaN,-0.333333,21.5,0.422222,7.455000,31.000000,NaN,NaN,10.500000,NaN,8.950000,112.000000,0.900000,NaN,106.500000,NaN,2.150000,1.800000,3.650000,NaN,NaN,30.966667,10.550000,36.600,17.350000,NaN,291.500000,27.24,1.0,0.035714
19965,83.267857,90.839286,36.912857,NaN,74.423077,NaN,17.482143,NaN,NaN,29.0,0.500000,NaN,NaN,NaN,NaN,14.000000,NaN,9.000000,100.000000,0.700000,NaN,171.000000,NaN,2.000000,2.900000,4.100000,NaN,NaN,44.000000,15.200000,32.900,5.400000,NaN,197.000000,27.86,1.0,0.034483


In [ ]:
cols = df_train.columns
stats = {}
for col in cols:
  stats[col]=[]
for patient in df_train["patient"].unique():
  pat_df = df_train[df_train["patient"]==patient]
  for col in cols:
    if pat_df[col].isna().sum()==pat_df.shape[0]:
      stats[col].append(patient)
print(stats)



In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score

# Load data
data = df_train

# Split data into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(
    data.drop('SepsisLabel', axis=1),
    data['SepsisLabel'],
    test_size=0.2,
    random_state=42
)

# Define XGBoost model parameters
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 3,
    'eta': 0.1
}

# Train XGBoost model
dtrain = xgb.DMatrix(X_train, label=y_train)
model = xgb.train(params, dtrain)

# Make predictions on test set
dtest = xgb.DMatrix(X_val)
y_pred = model.predict(dtest)

# Convert probabilities to binary predictions
y_pred_binary = [1 if p >= 0.5 else 0 for p in y_pred]

# Evaluate model accuracy
f1 = f1_score(y_val, y_pred_binary)
print(f'F1 score is: {f1:.2f}')